In [1]:
import optuna
import mlflow
import mlflow.sklearn
from pathlib import Path
import sqlite3
import pandas as pd
import sqlite3
from pathlib import Path


In [2]:
import os
import mlflow
from dotenv import load_dotenv

load_dotenv()  # loads .env file

mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))
mlflow.set_experiment(os.getenv("MLFLOW_EXPERIMENT_NAME"))

<Experiment: artifact_location='mlflow-artifacts:/56552c5a35a543eeb10fec244562cd11', creation_time=1766011374791, experiment_id='0', last_update_time=1766011374791, lifecycle_stage='active', name='Bike_Buyers_Classification', tags={}>

In [3]:
with mlflow.start_run(run_name="dagshub_test_env"):
    mlflow.log_metric("sanity_check_env", 1.0)

🏃 View run dagshub_test_env at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/92ff5972996e45e09ad6942e4fa1243c
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


In [6]:
from pathlib import Path
import sqlite3
import pandas as pd

BASE_DIR = Path("..")
DB_PATH = BASE_DIR / "data" / "bike_buyers.db"

In [7]:
conn = sqlite3.connect(DB_PATH)

tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)

conn.close()
tables

,name
0,sqlite_sequence
1,customers
2,education
3,occupation
4,housing
5,commute
6,region
7,purchases


In [8]:
conn = sqlite3.connect(DB_PATH)

query = """
SELECT
    c.customer_id,
    c.gender,
    c.age,
    c.marital_status,
    c.children,
    c.income,
    e.education_level,
    o.occupation_name,
    r.region_name,
    cm.commute_distance,
    h.home_owner,
    h.cars,
    p.purchased_bike
FROM purchases p
JOIN customers c ON p.customer_id = c.customer_id
JOIN education e ON p.education_id = e.education_id
JOIN occupation o ON p.occupation_id = o.occupation_id
JOIN region r ON p.region_id = r.region_id
JOIN commute cm ON p.commute_id = cm.commute_id
JOIN housing h ON p.housing_id = h.housing_id
"""

df = pd.read_sql(query, conn)
conn.close()

df.head(), df.shape

(   customer_id  gender  age marital_status  children  income  education_level  \
 0        12496  Female   42        Married         1   40000        Bachelors   
 1        24107    Male   43        Married         3   30000  Partial College   
 2        14177    Male   60        Married         5   80000  Partial College   
 3        24381    Male   41         Single         0   70000        Bachelors   
 4        25597    Male   36         Single         0   30000        Bachelors   
 
   occupation_name region_name commute_distance home_owner  cars  \
 0  Skilled Manual      Europe        0-1 Miles        Yes     0   
 1        Clerical      Europe        0-1 Miles        Yes     1   
 2    Professional      Europe        2-5 Miles         No     2   
 3    Professional     Pacific       5-10 Miles        Yes     1   
 4        Clerical      Europe        0-1 Miles         No     0   
 
    purchased_bike  
 0               0  
 1               0  
 2               0  
 3          

In [9]:
X = df.drop(columns=["purchased_bike", "customer_id"])
y = df["purchased_bike"]

X.shape, y.shape

((1000, 11), (1000,))

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

y_train.value_counts(normalize=True), y_test.value_counts(normalize=True)

(purchased_bike
 0    0.51875
 1    0.48125
 Name: proportion, dtype: float64,
 purchased_bike
 0    0.52
 1    0.48
 Name: proportion, dtype: float64)

In [11]:
categorical_features = X.select_dtypes(include="object").columns.tolist()
numeric_features = X.select_dtypes(exclude="object").columns.tolist()

categorical_features, numeric_features

(['gender',
  'marital_status',
  'education_level',
  'occupation_name',
  'region_name',
  'commute_distance',
  'home_owner'],
 ['age', 'children', 'income', 'cars'])

Preprocessing WITHOUT PCA

In [12]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

# Categorical & numeric transformers
categorical_transformer = OneHotEncoder(
    handle_unknown="ignore",
    sparse_output=False
)

numeric_transformer = StandardScaler()

# Preprocessor WITHOUT PCA
preprocessor_no_pca = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
        ("num", numeric_transformer, numeric_features)
    ]
)

Preprocessing WITH PCA

In [13]:
# Numeric pipeline WITH PCA
numeric_pca_pipeline = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=0.95, random_state=42))
    ]
)

# Preprocessor WITH PCA
preprocessor_with_pca = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
        ("num", numeric_pca_pipeline, numeric_features)
    ]
)

EXPERIMENT #3
Logistic Regression + ✅ PCA + ❌ No Optuna

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import mlflow

with mlflow.start_run(run_name="LogReg_PCA_NoOptuna"):
    logreg_pca_pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_with_pca),
            ("model", LogisticRegression(max_iter=1000, random_state=42))
        ]
    )
    
    logreg_pca_pipeline.fit(X_train, y_train)
    y_pred = logreg_pca_pipeline.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    
    # Log parameters & metric
    mlflow.log_param("model", "LogisticRegression")
    mlflow.log_param("pca", True)
    mlflow.log_param("optuna", False)
    mlflow.log_metric("f1_score", f1)
    
    f1

🏃 View run LogReg_PCA_NoOptuna at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/085c6b8b4d3f4271bd4ca3b1ae0391b6
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


EXPERIMENT #4
Logistic Regression + ✅ PCA + ✅ Optuna

In [15]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import mlflow

def objective_logreg_pca(trial):
    C = trial.suggest_float("C", 1e-3, 10.0, log=True)
    solver = trial.suggest_categorical("solver", ["liblinear", "lbfgs"])
    
    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_with_pca),
            ("model", LogisticRegression(
                C=C,
                solver=solver,
                max_iter=1000,
                random_state=42
            ))
        ]
    )
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    return f1_score(y_test, y_pred)

study = optuna.create_study(direction="maximize")
study.optimize(objective_logreg_pca, n_trials=20)

best_params = study.best_params
best_f1 = study.best_value

with mlflow.start_run(run_name="LogReg_PCA_Optuna"):
    mlflow.log_param("model", "LogisticRegression")
    mlflow.log_param("pca", True)
    mlflow.log_param("optuna", True)
    
    for k, v in best_params.items():
        mlflow.log_param(k, v)
    
    mlflow.log_metric("f1_score", best_f1)

best_f1

[I 2025-12-17 18:46:56,531] A new study created in memory with name: no-name-8c00b5a2-b2af-4c0f-8bc4-38df8cbd1729
[I 2025-12-17 18:46:56,551] Trial 0 finished with value: 0.6043956043956044 and parameters: {'C': 7.9384432611950615, 'solver': 'liblinear'}. Best is trial 0 with value: 0.6043956043956044.
[I 2025-12-17 18:46:56,608] Trial 1 finished with value: 0.6043956043956044 and parameters: {'C': 9.457159955152418, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.6043956043956044.
[I 2025-12-17 18:46:56,633] Trial 2 finished with value: 0.6120218579234973 and parameters: {'C': 0.6019186329906127, 'solver': 'liblinear'}. Best is trial 2 with value: 0.6120218579234973.
[I 2025-12-17 18:46:56,648] Trial 3 finished with value: 0.6120218579234973 and parameters: {'C': 0.6054926392938852, 'solver': 'liblinear'}. Best is trial 2 with value: 0.6120218579234973.
[I 2025-12-17 18:46:56,666] Trial 4 finished with value: 0.6120218579234973 and parameters: {'C': 0.8345314685042001, 'solver': 'li

🏃 View run LogReg_PCA_Optuna at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/8cae027676814248a8a25d17d7c2e7b9
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


0.6629834254143646

EXPERIMENT #1
Logistic Regression + ❌ PCA + ❌ Optuna

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import mlflow

with mlflow.start_run(run_name="LogReg_NoPCA_NoOptuna"):
    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_no_pca),
            ("model", LogisticRegression(max_iter=1000, random_state=42))
        ]
    )
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    
    mlflow.log_param("model", "LogisticRegression")
    mlflow.log_param("pca", False)
    mlflow.log_param("optuna", False)
    mlflow.log_metric("f1_score", f1)
    
    f1

🏃 View run LogReg_NoPCA_NoOptuna at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/c60daee1fbff49ec98d9e5f22edc2cc5
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


EXPERIMENT #2
Logistic Regression + ❌ PCA + ✅ Optuna

In [17]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import mlflow

def objective_logreg_no_pca(trial):
    C = trial.suggest_float("C", 1e-3, 10.0, log=True)
    solver = trial.suggest_categorical("solver", ["liblinear", "lbfgs"])
    
    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_no_pca),
            ("model", LogisticRegression(
                C=C,
                solver=solver,
                max_iter=1000,
                random_state=42
            ))
        ]
    )
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    return f1_score(y_test, y_pred)

study = optuna.create_study(direction="maximize")
study.optimize(objective_logreg_no_pca, n_trials=20)

best_params = study.best_params
best_f1 = study.best_value

with mlflow.start_run(run_name="LogReg_NoPCA_Optuna"):
    mlflow.log_param("model", "LogisticRegression")
    mlflow.log_param("pca", False)
    mlflow.log_param("optuna", True)
    
    for k, v in best_params.items():
        mlflow.log_param(k, v)
    
    mlflow.log_metric("f1_score", best_f1)

best_f1

[I 2025-12-17 18:47:06,831] A new study created in memory with name: no-name-f9795399-1ce1-4832-b452-aadc63f4fc6a
[I 2025-12-17 18:47:06,850] Trial 0 finished with value: 0.6120218579234973 and parameters: {'C': 1.0074298512941329, 'solver': 'liblinear'}. Best is trial 0 with value: 0.6120218579234973.
[I 2025-12-17 18:47:06,870] Trial 1 finished with value: 0.6043956043956044 and parameters: {'C': 2.722583619751426, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.6120218579234973.
[I 2025-12-17 18:47:06,890] Trial 2 finished with value: 0.6593406593406593 and parameters: {'C': 0.029856289554987682, 'solver': 'lbfgs'}. Best is trial 2 with value: 0.6593406593406593.
[I 2025-12-17 18:47:06,902] Trial 3 finished with value: 0.6206896551724138 and parameters: {'C': 0.003157091537650663, 'solver': 'liblinear'}. Best is trial 2 with value: 0.6593406593406593.
[I 2025-12-17 18:47:06,920] Trial 4 finished with value: 0.6440677966101694 and parameters: {'C': 0.0102318664119556, 'solver': 'lb

🏃 View run LogReg_NoPCA_Optuna at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/8312ab4e9e4a41dcaea4860d14639429
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


0.6629834254143646

EXPERIMENT #5
Gradient Boosting + ❌ PCA + ❌ Optuna

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
import mlflow

with mlflow.start_run(run_name="GB_NoPCA_NoOptuna"):
    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_no_pca),
            ("model", GradientBoostingClassifier(random_state=42))
        ]
    )
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    
    mlflow.log_param("model", "GradientBoosting")
    mlflow.log_param("pca", False)
    mlflow.log_param("optuna", False)
    mlflow.log_metric("f1_score", f1)
    
    f1

🏃 View run GB_NoPCA_NoOptuna at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/84788a4794e44884b3869ff3121f6835
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


EXPERIMENT #6
Gradient Boosting + ✅ PCA + ❌ Optuna

In [19]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
import mlflow

with mlflow.start_run(run_name="GB_PCA_NoOptuna"):
    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_with_pca),
            ("model", GradientBoostingClassifier(random_state=42))
        ]
    )
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    
    mlflow.log_param("model", "GradientBoosting")
    mlflow.log_param("pca", True)
    mlflow.log_param("optuna", False)
    mlflow.log_metric("f1_score", f1)
    
    f1

🏃 View run GB_PCA_NoOptuna at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/b5098709a006435e8ea444c4282378f4
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


EXPERIMENT #7
Gradient Boosting + ❌ PCA + ✅ Optuna

In [20]:
import optuna
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
import mlflow

def objective_gb_no_pca(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 5)

    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_no_pca),
            ("model", GradientBoostingClassifier(
                n_estimators=n_estimators,
                learning_rate=learning_rate,
                max_depth=max_depth,
                random_state=42
            ))
        ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    return f1_score(y_test, y_pred)

study = optuna.create_study(direction="maximize")
study.optimize(objective_gb_no_pca, n_trials=20)

best_params = study.best_params
best_f1 = study.best_value

with mlflow.start_run(run_name="GB_NoPCA_Optuna"):
    mlflow.log_param("model", "GradientBoosting")
    mlflow.log_param("pca", False)
    mlflow.log_param("optuna", True)

    for k, v in best_params.items():
        mlflow.log_param(k, v)

    mlflow.log_metric("f1_score", best_f1)

best_f1

[I 2025-12-17 18:47:36,438] A new study created in memory with name: no-name-e1076ed9-3989-4da0-9b02-aa7a31f4ae9b
[I 2025-12-17 18:47:36,599] Trial 0 finished with value: 0.6735751295336787 and parameters: {'n_estimators': 87, 'learning_rate': 0.07869604603489289, 'max_depth': 4}. Best is trial 0 with value: 0.6735751295336787.
[I 2025-12-17 18:47:36,965] Trial 1 finished with value: 0.6839378238341969 and parameters: {'n_estimators': 270, 'learning_rate': 0.014898817070108266, 'max_depth': 3}. Best is trial 1 with value: 0.6839378238341969.
[I 2025-12-17 18:47:37,168] Trial 2 finished with value: 0.7120418848167539 and parameters: {'n_estimators': 226, 'learning_rate': 0.07678410468888501, 'max_depth': 2}. Best is trial 2 with value: 0.7120418848167539.
[I 2025-12-17 18:47:37,405] Trial 3 finished with value: 0.6632653061224489 and parameters: {'n_estimators': 166, 'learning_rate': 0.012397244569018908, 'max_depth': 3}. Best is trial 2 with value: 0.7120418848167539.
[I 2025-12-17 18:

🏃 View run GB_NoPCA_Optuna at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/b9d8b6bdacd743a89715b4584f67344c
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


0.7127659574468085

EXPERIMENT #8
Gradient Boosting + ✅ PCA + ✅ Optuna

In [21]:
import optuna
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
import mlflow

def objective_gb_pca(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 5)

    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_with_pca),
            ("model", GradientBoostingClassifier(
                n_estimators=n_estimators,
                learning_rate=learning_rate,
                max_depth=max_depth,
                random_state=42
            ))
        ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    return f1_score(y_test, y_pred)

study = optuna.create_study(direction="maximize")
study.optimize(objective_gb_pca, n_trials=20)

best_params = study.best_params
best_f1 = study.best_value

with mlflow.start_run(run_name="GB_PCA_Optuna"):
    mlflow.log_param("model", "GradientBoosting")
    mlflow.log_param("pca", True)
    mlflow.log_param("optuna", True)

    for k, v in best_params.items():
        mlflow.log_param(k, v)

    mlflow.log_metric("f1_score", best_f1)

best_f1

[I 2025-12-17 18:47:45,981] A new study created in memory with name: no-name-606e7086-944a-43ae-a2f8-f08e0b0a7638
[I 2025-12-17 18:47:46,217] Trial 0 finished with value: 0.6486486486486487 and parameters: {'n_estimators': 187, 'learning_rate': 0.022483039701533546, 'max_depth': 2}. Best is trial 0 with value: 0.6486486486486487.
[I 2025-12-17 18:47:46,297] Trial 1 finished with value: 0.6666666666666666 and parameters: {'n_estimators': 58, 'learning_rate': 0.12949597058164763, 'max_depth': 2}. Best is trial 1 with value: 0.6666666666666666.
[I 2025-12-17 18:47:46,992] Trial 2 finished with value: 0.6593406593406593 and parameters: {'n_estimators': 249, 'learning_rate': 0.03182454179030414, 'max_depth': 5}. Best is trial 1 with value: 0.6666666666666666.
[I 2025-12-17 18:47:47,283] Trial 3 finished with value: 0.6702127659574468 and parameters: {'n_estimators': 214, 'learning_rate': 0.02756341546450541, 'max_depth': 2}. Best is trial 3 with value: 0.6702127659574468.
[I 2025-12-17 18:4

🏃 View run GB_PCA_Optuna at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/6e87b016d9bd42bb8885f1b7e2d71210
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


0.6739130434782609

EXPERIMENT #9
XGBoost + ❌ PCA + ❌ Optuna

In [22]:
import xgboost

In [24]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
import mlflow

with mlflow.start_run(run_name="XGB_NoPCA_NoOptuna"):
    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_no_pca),
            ("model", XGBClassifier(
                n_estimators=200,
                max_depth=4,
                learning_rate=0.1,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42,
                eval_metric="logloss"
            ))
        ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    mlflow.log_param("model", "XGBoost")
    mlflow.log_param("pca", False)
    mlflow.log_param("optuna", False)
    mlflow.log_metric("f1_score", f1)

    f1

🏃 View run XGB_NoPCA_NoOptuna at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/06c870614f4b4816b1f3ae3f457248b7
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


EXPERIMENT #10
XGBoost + ✅ PCA + ❌ Optuna

In [25]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
import mlflow

with mlflow.start_run(run_name="XGB_PCA_NoOptuna"):
    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_with_pca),
            ("model", XGBClassifier(
                n_estimators=200,
                max_depth=4,
                learning_rate=0.1,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42,
                eval_metric="logloss"
            ))
        ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    mlflow.log_param("model", "XGBoost")
    mlflow.log_param("pca", True)
    mlflow.log_param("optuna", False)
    mlflow.log_metric("f1_score", f1)

    f1

🏃 View run XGB_PCA_NoOptuna at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/251b403a008347acaf9b8d41af5515f5
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


EXPERIMENT #11
XGBoost + ❌ PCA + ✅ Optuna

In [26]:
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
import mlflow

def objective_xgb_no_pca(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 400),
        "max_depth": trial.suggest_int("max_depth", 3, 6),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "random_state": 42,
        "eval_metric": "logloss",
        "use_label_encoder": False
    }

    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_no_pca),
            ("model", XGBClassifier(**params))
        ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    return f1_score(y_test, y_pred)

study = optuna.create_study(direction="maximize")
study.optimize(objective_xgb_no_pca, n_trials=20)

best_params = study.best_params
best_f1 = study.best_value

with mlflow.start_run(run_name="XGB_NoPCA_Optuna"):
    mlflow.log_param("model", "XGBoost")
    mlflow.log_param("pca", False)
    mlflow.log_param("optuna", True)

    for k, v in best_params.items():
        mlflow.log_param(k, v)

    mlflow.log_metric("f1_score", best_f1)

best_f1

[I 2025-12-17 18:48:25,656] A new study created in memory with name: no-name-f713ee81-e733-4573-8b9f-3e774d082303
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [18:48:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[I 2025-12-17 18:48:25,770] Trial 0 finished with value: 0.6214689265536724 and parameters: {'n_estimators': 310, 'max_depth': 5, 'learning_rate': 0.06062638066219723, 'subsample': 0.7148814907250047, 'colsample_bytree': 0.829663615649688}. Best is trial 0 with value: 0.6214689265536724.
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [18:48:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not 

🏃 View run XGB_NoPCA_Optuna at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/3d2e1c92398c47349355d56c376ba3c0
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


0.7157894736842105

EXPERIMENT #12
XGBoost + ✅ PCA + ✅ Optuna

In [27]:
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
import mlflow

def objective_xgb_pca(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 400),
        "max_depth": trial.suggest_int("max_depth", 3, 6),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "random_state": 42,
        "eval_metric": "logloss",
        "use_label_encoder": False
    }

    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_with_pca),
            ("model", XGBClassifier(**params))
        ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    return f1_score(y_test, y_pred)

study = optuna.create_study(direction="maximize")
study.optimize(objective_xgb_pca, n_trials=20)

best_params = study.best_params
best_f1 = study.best_value

with mlflow.start_run(run_name="XGB_PCA_Optuna"):
    mlflow.log_param("model", "XGBoost")
    mlflow.log_param("pca", True)
    mlflow.log_param("optuna", True)

    for k, v in best_params.items():
        mlflow.log_param(k, v)

    mlflow.log_metric("f1_score", best_f1)

best_f1

[I 2025-12-17 18:48:35,534] A new study created in memory with name: no-name-6c95a9f9-3a00-4012-bc12-c984f70ff0a4
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [18:48:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[I 2025-12-17 18:48:35,622] Trial 0 finished with value: 0.6557377049180327 and parameters: {'n_estimators': 165, 'max_depth': 5, 'learning_rate': 0.03707395418175779, 'subsample': 0.7098060234511525, 'colsample_bytree': 0.673507012218107}. Best is trial 0 with value: 0.6557377049180327.
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [18:48:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not 

🏃 View run XGB_PCA_Optuna at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/607d9a49942b4e80be85143369b3ae12
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


0.6844919786096256

EXPERIMENT #13
LightGBM + ❌ PCA + ❌ Optuna

In [28]:
import lightgbm

In [29]:
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
import mlflow

with mlflow.start_run(run_name="LGBM_NoPCA_NoOptuna"):
    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_no_pca),
            ("model", LGBMClassifier(
                n_estimators=200,
                learning_rate=0.1,
                num_leaves=31,
                random_state=42
            ))
        ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    mlflow.log_param("model", "LightGBM")
    mlflow.log_param("pca", False)
    mlflow.log_param("optuna", False)
    mlflow.log_metric("f1_score", f1)

    f1

[LightGBM] [Info] Number of positive: 385, number of negative: 415
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000028 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 128
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.481250 -> initscore=-0.075035
[LightGBM] [Info] Start training from score -0.075035
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


🏃 View run LGBM_NoPCA_NoOptuna at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/27d8845d7db3410ea3b24ada41211157
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


EXPERIMENT #14
LightGBM + ✅ PCA + ❌ Optuna

In [30]:
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
import mlflow

with mlflow.start_run(run_name="LGBM_PCA_NoOptuna"):
    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_with_pca),
            ("model", LGBMClassifier(
                n_estimators=200,
                learning_rate=0.1,
                num_leaves=31,
                random_state=42
            ))
        ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    mlflow.log_param("model", "LightGBM")
    mlflow.log_param("pca", True)
    mlflow.log_param("optuna", False)
    mlflow.log_metric("f1_score", f1)

    f1

[LightGBM] [Info] Number of positive: 385, number of negative: 415
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000094 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1062
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.481250 -> initscore=-0.075035
[LightGBM] [Info] Start training from score -0.075035
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


🏃 View run LGBM_PCA_NoOptuna at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0/runs/956c4ecfe56e432ba7a8d623bc20e531
🧪 View experiment at: https://dagshub.com/harish-334/bike-buyers-mlflow.mlflow/#/experiments/0


EXPERIMENT #15
LightGBM + ❌ PCA + ✅ Optuna

In [32]:
import optuna
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
import mlflow

def objective_lgbm_no_pca(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 400),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 100),
        "max_depth": trial.suggest_int("max_depth", -1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "random_state": 42
    }

    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_no_pca),
            ("model", LGBMClassifier(**params))
        ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    return f1_score(y_test, y_pred)

study = optuna.create_study(direction="maximize")
study.optimize(objective_lgbm_no_pca, n_trials=20)

best_params = study.best_params
best_f1 = study.best_value

with mlflow.start_run(run_name="LGBM_NoPCA_Optuna"):
    mlflow.log_param("model", "LightGBM")
    mlflow.log_param("pca", False)
    mlflow.log_param("optuna", True)

    for k, v in best_params.items():
        mlflow.log_param(k, v)

    mlflow.log_metric("f1_score", best_f1)

best_f1

[I 2025-12-17 18:49:16,371] A new study created in memory with name: no-name-942e4654-838b-4c73-aef2-e09056554bd5
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:16,404] Trial 0 finished with value: 0.6703296703296703 and parameters: {'n_estimators': 117, 'learning_rate': 0.12288453223883705, 'num_leaves': 74, 'max_depth': 1, 'min_child_samples': 49, 'subsample': 0.8670275203634366, 'colsample_bytree': 0.7833835783313199}. Best is trial 0 with value: 0.6703296703296703.
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:16,453] Trial

[LightGBM] [Info] Number of positive: 385, number of negative: 415
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000090 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 128
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.481250 -> initscore=-0.075035
[LightGBM] [Info] Start training from score -0.075035
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:16,621] Trial 5 finished with value: 0.6595744680851063 and parameters: {'n_estimators': 375, 'learning_rate': 0.024175123317869647, 'num_leaves': 21, 'max_depth': 1, 'min_child_samples': 32, 'subsample': 0.9831879015270872, 'colsample_bytree': 0.6948814519156141}. Best is trial 3 with value: 0.6844919786096256.
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:16,673] Trial 6 finished with value: 0.6740331491712708 and parameters: {'n_estimators': 153, 'learning_rate': 0.0244608049311

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:16,802] Trial 8 finished with value: 0.6229508196721312 and parameters: {'n_estimators': 241, 'learning_rate': 0.2886187623139917, 'num_leaves': 65, 'max_depth': 7, 'min_child_samples': 44, 'subsample': 0.8511327190465614, 'colsample_bytree': 0.8042644158873962}. Best is trial 3 with value: 0.6844919786096256.
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:16,853] Trial 9 finished with value: 0.6810810810810811 and parameters: {'n_estimators': 161, 'learning_rate': 0.011973467966890

...

c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:17,486] Trial 16 finished with value: 0.6772486772486772 and parameters: {'n_estimators': 294, 'learning_rate': 0.17680907816149197, 'num_leaves': 32, 'max_depth': 3, 'min_child_samples': 28, 'subsample': 0.6990233869688979, 'colsample_bytree': 0.9493477217644614}. Best is trial 11 with value: 0.6918918918918919.
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:17,578] Trial 17 finished with value: 0.6593406593406593 and parameters: {'n_estimators': 333, 'learning_rate': 0.03553900761

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:17,667] Trial 18 finished with value: 0.644808743169399 and parameters: {'n_estimators': 280, 'learning_rate': 0.0783322648975092, 'num_leaves': 42, 'max_depth': 6, 'min_child_samples': 19, 'subsample': 0.8173295678875914, 'colsample_bytree': 0.7729216895740882}. Best is trial 11 with value: 0.6918918918918919.
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:17,744] Trial 19 finished with value: 0.6404494382022472 and parameters: {'n_estimators': 326, 'learning_rate': 0.1994407537032

[LightGBM] [Info] Number of positive: 385, number of negative: 415
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000091 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 128
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.481250 -> initscore=-0.075035
[LightGBM] [Info] Start training from score -0.075035
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

0.6918918918918919

EXPERIMENT #16
LightGBM + ✅ PCA + ✅ Optuna

In [33]:
import optuna
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
import mlflow

def objective_lgbm_pca(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 400),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 100),
        "max_depth": trial.suggest_int("max_depth", -1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "random_state": 42
    }

    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor_with_pca),
            ("model", LGBMClassifier(**params))
        ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    return f1_score(y_test, y_pred)

study = optuna.create_study(direction="maximize")
study.optimize(objective_lgbm_pca, n_trials=20)

best_params = study.best_params
best_f1 = study.best_value

with mlflow.start_run(run_name="LGBM_PCA_Optuna"):
    mlflow.log_param("model", "LightGBM")
    mlflow.log_param("pca", True)
    mlflow.log_param("optuna", True)

    for k, v in best_params.items():
        mlflow.log_param(k, v)

    mlflow.log_metric("f1_score", best_f1)

best_f1

[I 2025-12-17 18:49:35,799] A new study created in memory with name: no-name-071b79e7-9917-4d26-a008-83531957bd4d
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:35,861] Trial 0 finished with value: 0.6408839779005525 and parameters: {'n_estimators': 216, 'learning_rate': 0.10090405085555038, 'num_leaves': 72, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.6724012358279333, 'colsample_bytree': 0.8264256125087949}. Best is trial 0 with value: 0.6408839779005525.
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:36,005] Trial

[LightGBM] [Info] Number of positive: 385, number of negative: 415
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000074 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1062
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.481250 -> initscore=-0.075035
[LightGBM] [Info] Start training from score -0.075035
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:36,048] Trial 2 finished with value: 0.6413043478260869 and parameters: {'n_estimators': 124, 'learning_rate': 0.16046844016565082, 'num_leaves': 26, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.7372786416749152, 'colsample_bytree': 0.6386538705227127}. Best is trial 2 with value: 0.6413043478260869.
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:36,115] Trial 3 finished with value: 0.6451612903225806 and parameters: {'n_estimators': 218, 'learning_rate': 0.03853804037183

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:36,231] Trial 4 finished with value: 0.632768361581921 and parameters: {'n_estimators': 185, 'learning_rate': 0.10612205488961715, 'num_leaves': 96, 'max_depth': 6, 'min_child_samples': 35, 'subsample': 0.9001252250096567, 'colsample_bytree': 0.7679360544082359}. Best is trial 3 with value: 0.6451612903225806.
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:36,293] Trial 5 finished with value: 0.6451612903225806 and parameters: {'n_estimators': 135, 'learning_rate': 0.038471897625922

...

c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:37,112] Trial 15 finished with value: 0.6736842105263158 and parameters: {'n_estimators': 327, 'learning_rate': 0.019798197745792796, 'num_leaves': 55, 'max_depth': 1, 'min_child_samples': 44, 'subsample': 0.6559466968261463, 'colsample_bytree': 0.881665236920656}. Best is trial 15 with value: 0.6736842105263158.
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:37,170] Trial 16 finished with value: 0.6666666666666666 and parameters: {'n_estimators': 327, 'learning_rate': 0.02354096592

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:37,381] Trial 18 finished with value: 0.6486486486486487 and parameters: {'n_estimators': 390, 'learning_rate': 0.010476590470798401, 'num_leaves': 33, 'max_depth': -1, 'min_child_samples': 42, 'subsample': 0.6552036818324628, 'colsample_bytree': 0.8493859590742205}. Best is trial 15 with value: 0.6736842105263158.
c:\Users\haris\Desktop\Sem 1\EAS 503 Prog DB\Final Project\bike_buyers_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-12-17 18:49:37,431] Trial 19 finished with value: 0.6666666666666666 and parameters: {'n_estimators': 328, 'learning_rate': 0.018145947

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

0.6736842105263158

# Highest F1-score visible:
# GradientBoosting + No PCA + Optuna
# f1 ≈ 0.7196

In [34]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from pathlib import Path
import joblib

In [35]:
final_model = Pipeline(
    steps=[
        ("preprocessor", preprocessor_no_pca),
        ("model", GradientBoostingClassifier(
            n_estimators=200,
            learning_rate=0.1,
            max_depth=3,
            random_state=42
        ))
    ]
)

final_model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat', ...), ('num', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers co

In [37]:
from pathlib import Path
import joblib

MODEL_PATH = Path("..") / "models" / "global_best_model.pkl"
MODEL_PATH.parent.mkdir(exist_ok=True)

joblib.dump(final_model, MODEL_PATH)

MODEL_PATH

WindowsPath('../models/global_best_model.pkl')